In [1]:
using DrWatson
@quickactivate "dare"

using ReinforcementLearning
using IntervalSets
using LinearAlgebra
using ControlSystems
using CUDA
using DataFrames
using Plots
using Interpolations


include(srcdir("nodeconstructor.jl"))
include(srcdir("LookupTables.jl"))
include(srcdir("env.jl"));

using .LookupTables

In [2]:
Base.@kwdef mutable struct battery_module
     R = [0.010342 0.0012244;
        0.0067316 0.0011396;
        0.0051156 0.0012661;
        0.0043447 0.0012265;
        0.0038826 0.0011163;
        0.0034226 0.0009968;
        0.003346 0.0011458;
        0.0033222 0.001345;
        0.0033201 0.0013091;
        0.0032886 0.0010986;
        0.0028114 0.0010309]     # Resistance for n-th RC pair
    C = [2287.7 11897;
        6122 24515;
        18460 42098;
        20975 44453;
        15254 33098;
        10440 24492;
        13903 32975;
        16694 40007;
        15784 35937;
        12165 26430;
        9118 24795]   # Capacitance for n-th RC pair
end;

In [3]:
Base.@kwdef mutable struct battery_block
    battery_module::battery_module
    n = 3                      # number of RC pairs
    R_0 = 0.016                  # series resistance
    V_0 = [3.5042 3.5136;
        3.5573 3.5646;
        3.6009 3.6153;
        3.6393 3.6565;
        3.6742 3.6889;
        3.7121 3.7214;
        3.7937 3.8078;
        3.8753 3.8945;
        3.97 3.9859;
        4.0764 4.0821;
        4.1924 4.193]                  # Battery open-circuit voltage
    Q = 0                      # Current Ladung
    Q_0 = 26*3600              # Nominal Ladung
    SOC = 0                    # State of Charge (SOC)
    SOC_BP = vec([0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1])
    T_BP = vec([293.15 313.15])
    tau = 1 #0.1e-3
    mode = nothing
    V = 0
    V1 = 0
    LT_V0 = LookupTable2d(SOC_BP,T_BP,V_0)
    LT_R = LookupTable2d(SOC_BP,T_BP,battery_module.R)
    
end;

In [4]:
function update(self::battery_block, I_batt)
    
    T=293.15 + 10
    
    if sign(I_batt) == -1
        self.mode = "charge"
    else
        self.mode = "discharge"
    end
    
    self.Q = clamp(self.Q -I_batt*self.tau,0,self.Q_0)
        
    self.SOC = clamp(self.Q/self.Q_0,0,1)
    
    SOC_round = round(self.SOC;digits = 1)
    
    idx_soc = findall(x->x==SOC_round, self.SOC_BP)[1]
    
    idx_t = 1
    
    self.V = self.V_0[idx_soc, idx_t] - self.n * self.battery_module.R[idx_soc,idx_t] * I_batt - self.R_0 * I_batt
    
    self.V1 = lookup(self.SOC,T,self.LT_V0) - self.n * lookup(self.SOC,T,self.LT_R) * I_batt - self.R_0 * I_batt
    
end

update (generic function with 1 method)

In [5]:
Cell = battery_module();

In [6]:
Battery = battery_block(battery_module=Cell);

In [8]:
for i in 1:10
    update(Battery, -400.0)
    
    println("Mode: $(Battery.mode)")
    println("State of Charge $(Battery.SOC)")
    println("Output Voltage $(Battery.V)")
    println("Output Voltage $(Battery.V1)")
end

Mode: charge
State of Charge 0.02564102564102564
Output Voltage 22.3146
Output Voltage 16.293593846153847
Mode: charge
State of Charge 0.029914529914529916
Output Voltage 22.3146
Output Voltage 16.20106948717949
Mode: charge
State of Charge 0.03418803418803419
Output Voltage 22.3146
Output Voltage 16.10854512820513
Mode: charge
State of Charge 0.038461538461538464
Output Voltage 22.3146
Output Voltage 16.01602076923077
Mode: charge
State of Charge 0.042735042735042736
Output Voltage 22.3146
Output Voltage 15.92349641025641
Mode: charge
State of Charge 0.04700854700854701
Output Voltage 22.3146
Output Voltage 15.830972051282052
Mode: charge
State of Charge 0.05128205128205128
Output Voltage 18.03522
Output Voltage 15.738447692307693
Mode: charge
State of Charge 0.05555555555555555
Output Voltage 18.03522
Output Voltage 15.645923333333334
Mode: charge
State of Charge 0.05982905982905983
Output Voltage 18.03522
Output Voltage 15.553398974358975
Mode: charge
State of Charge 0.0641025641025